# Visual Odometry for KITTI Dataset

Implementation of VO

## Google Colab


In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')
# %cd '/content/drive/MyDrive/Colab Notebooks'

# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# import math
# import scipy.signal
# import scipy.linalg
# import importlib
# import mae6292.tools as mae6292

# from google.colab.patches import cv2_imshow
# from google.colab import files as FILE
# import os

## Jupyter Notebook

In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import scipy.signal
import scipy.linalg
import mae6292.tools as mae6292
import importlib

from mae6292.imshow import cv2_imshow

## Boostrapping

Bootstrapping to initilize VO has been implemented by
```
keypoints0, p_W0, R1, T1 = mae6292.VO_bootstrap(img0, img1, K, display = False)   
```

In [47]:
importlib.reload(mae6292)

K = np.loadtxt('data/K.txt')
img0 = cv2.imread('data/000000.png', cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread('data/000001.png', cv2.IMREAD_GRAYSCALE)

# PARAMETERS
param_bootstrap = {
    # keypoints
    'W_harris_patch' : 4, # size of harris patch
    'kappa_harris' : 0.08, # kappa for harris score
    'N_keypoint' : 2000, # number of keypoints to be detected
    'W_nms' : 8, # patch size for non-maximum supression
    # KLT
    'W_KLT' : 4, # patch size for KLT
    'tol_KLT_bidir' : 1, # tolerence of bidirectional error
    # find essential matrix
    'tol_E' : 1, # tolerence for epipolar line distance
    'tol_E_RANSAC_prob' : 0.99, # eseential matrix RANSAC probability
    # triangulation
    'tol_TRI_mu' : 1e-3, # tolerence for the singular value ratio
    'tol_TRI_rep' : 1 # tolerence for the reprojection error
}

keypoints0, p_W0, R1, T1 = mae6292.VO_bootstrap(img0, img1, K, param_bootstrap, display = True)   
scale = 0.693071855459021
T1 = T1*scale
p_W0 = p_W0*scale

print('N_keypoints0=',len(keypoints0))


N_keypoints0= 851


## Localization and Mapping


The sequential process for localization and mapping has been implemented by
```
    R, T, S, C, fig = mae6292.VO_localization_mapping(i_frame, K, img, img_pre, S_pre, C_pre, display_process=True)
```
Each frame is saved under the folder `output`

In [59]:
#open a new window for plot
%matplotlib tk 
importlib.reload(mae6292)

# PARAMETERS
param = {
    # keypoints
    'W_harris_patch' : 4, # size of harris patch
    'kappa_harris' : 0.08, # kappa for harris score
    'N_keypoint' : 2000, # number of keypoints to be detected
    'W_nms' : 8, # patch size for non-maximum supression
    # KLT
    'W_KLT' : 4, # patch size for KLT
    'tol_KLT_bidir' : 1, # tolerence of bidirectional error
    # triangulation
    'tol_TRI_mu' : 1e-3, # tolerence for the singular value ratio
    'tol_TRI_rep' : 0.5, # tolerence for the reprojection error
    # mapping
    'tol_keypoints_new' : 20 # new keypoints should be district from the tracked keypoints by this distance
}

# iniitlize iteration
img_pre = img0
S_pre = mae6292.state(keypoints0, p_W0, [np.zeros((3,1))])
C_pre = mae6292.candidate([],[],[],[])

# varialbes to save the vehicle location and the keypoints in the W-frame 
T_W = np.zeros((3,N_frames))
p_W = p_W0

# number of frames to process
N_frames = 10
# boolean 
display_process = False

for i_frame in range(1,N_frames):
    
    print('i_frame=',i_frame)

    # VO localization and mapping
    img = cv2.imread("data/{:06d}.png".format(i_frame),cv2.IMREAD_GRAYSCALE)
    R, T, S, C, fig = mae6292.VO_localization_mapping(i_frame, img, img_pre, S_pre, C_pre, K, param)
    img_pre, S_pre, C_pre = img, S, C

    # save figure 
    if display_process:
        fig.savefig("output/{:06d}.png".format(i_frame))

    # save the vehicle location and the distinct keypoints 
    T_W[:,i_frame] = (-R.T@T).flatten()
    p_W_dist = scipy.spatial.distance.cdist( S.p_W.T, p_W.T , 'euclidean')
    index_distinct = np.where( np.min(p_W_dist, axis=1) > 3 )[0]
    p_W = np.append(p_W, S.p_W[:,index_distinct], axis=1)

    # print pose
    print('R=',R)
    print('T_W=',(-R.T@T).flatten())
    print(' ')




i_frame= 1
R= [[ 0.99999267  0.00327993  0.00197406]
 [-0.00328373  0.99999276  0.00192176]
 [-0.00196774 -0.00192822  0.9999962 ]]
T_W= [-0.01356089 -0.00294668  0.69992327]
 
i_frame= 2
S_pre= 851 , KLT_matched= 671 , PnP_inliers= 669 , S_new =  901
C_pre= 593 , KLT_matched= 415 , TRI_inliers= 232 , C_new =  333
R= [[ 0.99998542  0.00211629  0.0049686 ]
 [-0.00213458  0.99999096  0.00367715]
 [-0.00496077 -0.00368771  0.9999809 ]]
T_W= [-3.65094029e-02  2.02511934e-04  1.43034279e+00]
 
i_frame= 3
S_pre= 901 , KLT_matched= 793 , PnP_inliers= 787 , S_new =  823
C_pre= 333 , KLT_matched= 169 , TRI_inliers= 36 , C_new =  374
R= [[ 0.99996371  0.00219175  0.00823262]
 [-0.00223501  0.99998372  0.00524964]
 [-0.00822098 -0.00526785  0.99995233]]
T_W= [-5.82628984e-02  2.14676680e-03  2.18313254e+00]
 
i_frame= 4
S_pre= 823 , KLT_matched= 721 , PnP_inliers= 692 , S_new =  711
C_pre= 374 , KLT_matched= 167 , TRI_inliers= 19 , C_new =  372
R= [[ 9.99930615e-01  6.94619468e-04  1.17593529e-02

The following codes visualize the vehicle trajectory and the keypoints in the W-frame. 

(You may need to adjust th eaxis limit)

In [63]:

fig = plt.figure()
ax = plt.axes(projection = '3d')
ax.plot(T_W[0,:], T_W[1,:], T_W[2,:], 'b')
ax.scatter(p_W[0,:], p_W[1,:], p_W[2,:], s=1, c='r', marker='o')
ax.set_xlim(-10,20)
ax.set_zlim(-2,20)
ax.set_xlabel('x',fontsize=6)
ax.set_ylabel('y',fontsize=6)
ax.set_zlabel('z',fontsize=6)
ax.view_init(elev=0., azim=-90)

### Video

The output images can be converted into a video as follows.

In [18]:
img_array = []

for i_frame in range(1,N_frames):
    filename = "output/{:06d}.png".format(i_frame)
    img = cv2.imread(filename)
    img_array.append(img)

height, width, layers = img.shape
size = (width,height)

fps = 3
codec = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4',codec, fps, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])

out.release()